# Setup

In [10]:
#! pip install pandas-datareader
#! pip install pmdarima
#! pip install plotly
import numpy as np
from pandas_datareader import DataReader # pip install pandas-datareader
from pandas_datareader import data
from datetime import datetime
from pmdarima.arima import *
from pmdarima import preprocessing
from scipy import stats
from scipy.stats import skew

from statsmodels import api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

import pandas as pd
import pmdarima as pm
import plotly.graph_objects as go
import plotly.express as px
 
start = pd.to_datetime('2010-01-01') # in YYYY-MM-DD format
end = pd.to_datetime('2018-01-01')
ts = data.DataReader('NDAQ', 'yahoo', start , end) # here 'yahoo' is the API to yahoo

# Normalisation

In [5]:
# put all normalisation functions before modelling

In [22]:
#transformed data
boxCoxData, boxCox_lambda = stats.boxcox(ts.Close)
boxCoxSkew = skew(boxCoxData)

#we want to compare their absolute skewness 
if(boxCoxSkew < 0):
    boxCoxSkew = boxCoxSkew * -1

johnsonData, johnson_lambda = stats.yeojohnson(ts.Close)
johnsonSkew = skew(johnsonData)

if(johnsonSkew < 0):
    johnsonSkew = johnsonSkew * -1

#to show which is less skewed 
print(johnsonSkew)
print(boxCoxSkew)

0.043066305774378665
0.0391631256848633


In [23]:
if(johnsonSkew < boxCoxSkew):
    normalised_Data = johnsonData
    fitted_lambda = johnson_lambda
    
#being == doesn't really matter same either way
elif(boxCoxSkew <= johnsonSkew):
    normalised_Data = boxCoxData
    fitted_lambda = boxCox_lambda


skew(normalised_Data)
#print(fitted_lambda)

0.0391631256848633

In [ ]:
fig = px.line(ts.Close, x=[ts.index], y="Close")
fig.show()

# Modelling Functions

## Stationarity Tests:

In [ ]:
def testLevelStationarity(ts):
    d = ndiffs(ts, test='kpss')
    d += ndiffs(ts, test='adf')
    d += ndiffs(ts, test='pp')
    
    return int(d/3)

## Seasonal Tests:

In [ ]:
def seasonal_tests(data):
     
    result1 = pm.arima.nsdiffs(data, m=12, max_D=12, test='ch')
    
    print("CH results: " + str(result1))
    
    result2 = pm.arima.nsdiffs(data, m=12, max_D=12, test='ocsb')
    
    print("OCSB results: " + str(result2))
    
    return int((result1+result2)/2)


In [ ]:
seasonal_tests(ts.Close)

## Lag Period:

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.DataFrame (diff,columns=['Difference'])

In [ ]:
def find_lag_period(data):
    
    fig = px.line(ts.Close, x=[ts.index], y="Close")
    fig.add_vline(x='2010-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2011-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2012-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2013-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2014-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2015-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2016-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2017-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2018-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.show()
    
    adf_test = ADFTest(alpha = 0.05)
    
    test = list()
    lags = [365, 182, 90, 30, 14]
    
    # original data
    test.append(adf_test.should_diff(data))
    print(test[0])
    
    # year lag
    test.append(adf_test.should_diff(difference(data, 365)))
    print(test[1])
    
    # six months lag
    test.append(adf_test.should_diff(difference(data, 182))) 
    print(test[2])
    
    # three months lag 
    test.append(adf_test.should_diff(difference(data, 90)))
    print(test[3])
    
    # one month lag 
    test.append(adf_test.should_diff(difference(data, 30)))
    print(test[4])
    
    # fortnight lag
    test.append(adf_test.should_diff(difference(data, 14))) 
    print(test[5])
    
    # finds the first lag that doesn't need to be differenced
    for i in range(6):
        if (test[i][1] == False):
            return int(lags[i])
        
    return -1

In [ ]:
print(find_lag_period(ts.Close))


## AR & MA Tests

In [ ]:
def getAicBicHqic(dataset, arimaOrder, seasonalOrder):
    
    try:
        model = sm.tsa.statespace.SARIMAX(dataset, order = arimaOrder, seasonal_order=seasonalOrder).fit(disp=False)

        aic = model.aic
        bic = model.bic
        hqic = model.hqic
        
    
    except:
        pass
    
    return aic, bic, hqic

In [ ]:
def evaluateSarimaModels(dataset, pVals, dVal, qVals, seasonalPVals, seasonalDval, seasonalQVals, m):
    
    model = sm.tsa.statespace.SARIMAX(dataset, order = (0,0,0), seasonal_order=(0,0,0,m)).fit(disp=False)

    bestAic = model.aic
    bestBic = model.bic
    bestHqic = model.hqic
    
    for p in pVals:
        for q in qVals:
            for seasonalP in seasonalPVals:
                for seasonalQ in seasonalQVals:
                    order=(p,dVal,q)
                    seasonalOrder = (seasonalP, seasonalDval, seasonalQ, m)
                    try:
                        aic, bic, hqic = getAicBicHqic(dataset, order, seasonalOrder)
                        if(((aic < bestAic) and (bic<bestBic)) or ((aic < bestAic) and (hqic<bestHqic)) or ((bic<bestBic) and (hqic<bestHqic))):
                            bestAic = aic
                            bestBic = bic
                            bestHqic = hqic
                            bestOrder = order
                            bestSeasonalOrder = seasonalOrder
                        
                    except:
                        continue
        print(bestOrder, bestSeasonalOrder, bestAic, bestBic, bestHqic)
        return bestOrder, bestSeasonalOrder

In [ ]:
order, seasonalOrder = evaluateSarimaModels(ts.Close, [0,1], 1, [0,1], [0,1], 1, [0,1], 12)

In [ ]:
def sarima_model (data):
#train_test_split
    level_diff = testLevelStationarity(ts.Close)
    
    stepwise_model_7 = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=level_diff, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    
    return stepwise_model_7

In [ ]:
model_7 = sarima_model(ts.Close)

In [ ]:
normalised_stepwise_model_7 = auto_arima(normalised_Data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

In [ ]:
train = ts.Close.loc['2010-01-01':'2018-01-01']
test_7 = ts.Close.loc['2017-12-20': '2018-01-01']
test_31 = ts.Close.loc['2017-11-15':'2018-01-01']
two_month = ts.Close.loc['2017-10-15':'2018-01-01']


In [ ]:
print (normali)

In [ ]:
fig = px.line(future_forecast_7, x=test_31.index, y=future_forecast_7)
fig.add_scatter(x=test_31.index, y=test_31.values, mode='lines')
fig.show()

In [ ]:
test = sm.tsa.statespace.SARIMAX(normalised_Data, order = (1,1,1), seasonal_order=(1,1,1,12)).fit(disp=True)

In [ ]:
get_pred = test.get_prediction(0,len(normalised_Data),dynamic=False,full_results=True)

In [ ]:
get_pred.conf_int(0.5)

In [ ]:
normalised_future_forecast

In [ ]:
test_31

In [ ]:
normalised_Data

In [ ]:
#reverse
#restored_Data = (normalised_future_forecast*fitted_lambda +1)**(1/fitted_lambda)

In [ ]:
restored_Data

In [ ]:
fig = px.line(normalised_future_forecast, x=test_31.index, y=normalised_future_forecast)
fig.add_scatter(x=test_31.index, y=test_31.values, mode='lines')
fig.show()

In [ ]:
#evaluation of model fit
#boxljung test
#model_df is p+q
#m is seasonal period
#change boxpierce to true if you want to also run that test
#lbVal is the Ljung-Box test statistic and pVal is its p value
def getBoxLjung(data, lags, model_df, m):
    #demean the data
    demeanedData = data.sub(data.mean())
    lbVal, pVal = statsmodels.stats.diagnostic.acorr_ljungbox(demeanedData, lags, boxpierce = False, model_df = model_df, period = m)
    
    return lbVal, pVal
    

In [ ]:
#Pass a fitted model
#returns 
#JBVal = Jarque-Bera test statistic
#JBPVal = pvalue of the test statistic
def getJarqueBera(data):
    
    JBVal, JBPVal, skewness, kurtosis = statsmodels.stats.stattools.jarque_bera(data.resid)
    
    return JBVal, JBPVal, skewness, kurtosis

# Function to sort different models


In [ ]:
import csv


#input is a dataframe full of models
#I assume that there is no column that represents the skewness of model
def sort_models(models):
    skewness = []
    #loop to get summary table of every model, then get skewness
    
    #To iterate through the df and isolate the model
    for index,rows in models.iterrows():
        #wrong parameters, 
        summary_table = SARIMAXResults.summary(row['model'])
        table_csv = summary_table.as_csv();

        with open(table_csv, newline='') as myFile:
            #reader object stores all data
            reader = csv.reader(myFile)
            #change to finds out what cell skewness is in
            skewness.append(reader[2][3])
    #at this point, skewness array should be full
    models['skewness'] = skewness
    return models;
    
    